<a href="https://colab.research.google.com/github/YianKim/2022_uncertainty_aware_semisupervise/blob/main/Keras_UncertaintyBootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 7.9 MB/s 


In [7]:
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import clone_model

import PIL
from PIL import Image

import pickle
import random
from tqdm import tqdm
from collections import Counter

from keras.layers.core import Lambda
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D, AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras import optimizers
from keras.callbacks import *
from sklearn.metrics import *
from keras.models import load_model
import tensorflow_addons as tfa

from torchvision import transforms

import tensorflow as tf
import tensorflow.keras.backend as backend
import math
import gc

# SVHN

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
from scipy.io import loadmat
train_raw = loadmat('/content/drive/MyDrive/SVHN/train_32x32.mat')
test_raw = loadmat('/content/drive/MyDrive/SVHN/test_32x32.mat')

In [8]:
def dummy_labels(labels):
  zero_labels = np.zeros([labels.shape[0], 10], np.int8)  
  for i in range(labels.shape[0]):
    zero_labels[i][labels[i]] = 1
  return(zero_labels)

In [11]:
train_images = train_raw['X']
train_labels = train_raw['y']

test_images = test_raw['X']
test_labels = dummy_labels(test_raw['y']-1)

train_images = train_images.swapaxes(2,3).swapaxes(1,2).swapaxes(0,1)
test_images = test_images.swapaxes(2,3).swapaxes(1,2).swapaxes(0,1)

In [12]:
temp = [0,0,0,0,0,0,0,0,0,0]
label_indx = []
unlabel_indx = []

for i in range(73257) :
  if temp[(train_labels).reshape([-1])[i]-1] < 25 :
    temp[(train_labels).reshape([-1])[i]-1] += 1
    label_indx.append(i)
  else :
    unlabel_indx.append(i)

In [13]:
lbl_train_images = train_images[label_indx]
lbl_train_labels = dummy_labels(train_labels[label_indx]-1)

In [14]:
ubl_train_images = train_images[unlabel_indx]
ubl_train_labels = dummy_labels(train_labels[unlabel_indx]-1)

# CIFAR 10

In [ ]:
cifar10 = keras.datasets.cifar10 
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = train_images/255
test_images = test_images/255

In [ ]:
def dummy_labels(labels):
  zero_labels = np.zeros([labels.shape[0], 10], np.int8)  
  for i in range(labels.shape[0]):
    zero_labels[i][labels[i]] = 1
  return(zero_labels)

In [ ]:
train_labels = dummy_labels(train_labels)
test_labels = dummy_labels(test_labels)

In [ ]:
# 1000 labeled, 49000 unlabeled
random.seed(10)
indx = random.sample(range(train_labels.shape[0]),train_labels.shape[0])

lbl_train_images = train_images[indx[:1000]]
ubl_train_images = train_images[indx[1000:]]

lbl_train_labels = train_labels[indx[:1000]]
ubl_train_labels = train_labels[indx[1000:]]

# valids1 =  train_images[indx[800:1000]]
# valids2 =  train_labels[indx[800:1000]]

#MAin


In [15]:
class SGDR(Callback):

    def __init__(self, min_lr=0.0, max_lr=0.03, base_epochs=20, mul_epochs=2):
        super(SGDR, self).__init__()

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.base_epochs = base_epochs
        self.mul_epochs = mul_epochs

        self.cycles = 0.
        self.cycle_iterations = 0.
        self.trn_iterations = 0.

        self._reset()

    def _reset(self, new_min_lr=None, new_max_lr=None,
               new_base_epochs=None, new_mul_epochs=None):
        """Resets cycle iterations."""
        
        if new_min_lr != None:
            self.min_lr = new_min_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_base_epochs != None:
            self.base_epochs = new_base_epochs
        if new_mul_epochs != None:
            self.mul_epochs = new_mul_epochs
        self.cycles = 0.
        self.cycle_iterations = 0.
        
    def sgdr(self):
        
        cycle_epochs = self.base_epochs * (self.mul_epochs ** self.cycles)
        tide = ((self.cycles == 0) * 1) * (self.cycle_iterations*self.max_lr + (self.base_epochs - self.cycle_iterations)*self.min_lr) / self.base_epochs + ((self.cycles != 0) * 1)*(self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(np.pi * (self.cycle_iterations + 1) / cycle_epochs)))
        return tide
        
    def on_train_begin(self, logs=None):
        
        if self.cycle_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.max_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.sgdr())
            
    def on_epoch_end(self, epoch, logs=None):
        
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)
        
        self.trn_iterations += 1
        self.cycle_iterations += 1
        if self.cycle_iterations >= self.base_epochs * (self.mul_epochs ** self.cycles):
            self.cycles += 1
            self.cycle_iterations = 0
            K.set_value(self.model.optimizer.lr, self.max_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.sgdr())

In [16]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

def create_cnn_13():
  inputlayer = keras.Input(shape=(32, 32, 3))

  conv1a = Conv2D(128, (5,5), padding = 'same')
  bn1a = BatchNormalization()
  conv1b = Conv2D(64, (5,5), padding = 'same')
  bn1b = BatchNormalization()
  conv1c = Conv2D(32, (5,5), padding = 'same')
  bn1c = BatchNormalization()
  pl1 = MaxPooling2D(2, 2)
  MCdrop1 = PermaDropout(0.5)

  fc1 = Dense(1024)
  fc2 = Dense(10)
  activ = keras.layers.LeakyReLU(0.1)

  model = Sequential([
                  inputlayer, 
                  tfa.layers.WeightNormalization(conv1a), bn1a, activ, pl1,
                  tfa.layers.WeightNormalization(conv1b), bn1b, activ, pl1,
                  tfa.layers.WeightNormalization(conv1c), bn1c, activ, pl1,
                  MCdrop1, Flatten(),

                  fc1, fc2
                  ])
  
  return model

def compile_cnn_13(model):

  opt = keras.optimizers.SGD(0.03, momentum=0.9)

  model.compile(
    optimizer = opt,
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
  )

  return model

def cnn_13():

  model = create_cnn_13()
  model = compile_cnn_13(model)

  return model

def fit_and_labeling_cnn_13(Epoch, Batch):

  lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
  early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, mode='auto')
  sgdr = SGDR(min_lr=0.0, max_lr=0.03, base_epochs=20) #스케줄러

  model.fit(
      x=X,
      y=y,
      epochs=Epoch,
      verbose=1,
      batch_size=Batch,
      callbacks=[sgdr]
  )
    
  model_test_eval(model, test_images, test_labels)
  T = 1

  for predsamples in (range(10)):
    if predsamples == 0 :
      predictions = np.array(tf.nn.softmax(model.predict(ubl_train_images)/T))
      predictions = predictions.reshape((1,) + predictions.shape)
    else:
      pred = np.array(tf.nn.softmax(model.predict(ubl_train_images)/T))
      pred = pred.reshape((1,) + pred.shape)
      predictions = np.concatenate((predictions, pred))

  return predictions

def model_test_eval(model, test_images, test_labels):
  T = 1
  pred = np.array(tf.nn.softmax(model.predict(test_images)/T))
  for i in range(1,10):
    pred += np.array(tf.nn.softmax(model.predict(test_images)))
  acc = (np.argmax(pred,axis=1) == np.argmax(test_labels,axis=1))*1
  acc = sum(acc)/len(acc)
  print("test set 성능 : " + str(acc))

In [17]:
model = cnn_13()
X = lbl_train_images
y = lbl_train_labels
predictions = fit_and_labeling_cnn_13(500, 64)

Epoch 1/500
4/4 [==============================] - 12s 75ms/step - loss: 4.9665 - accuracy: 0.1080 - lr: 0.0300
Epoch 2/500
4/4 [==============================] - 0s 11ms/step - loss: 5.0718 - accuracy: 0.0920 - lr: 0.0015
Epoch 3/500
4/4 [==============================] - 0s 11ms/step - loss: 5.3698 - accuracy: 0.1120 - lr: 0.0030
Epoch 4/500
4/4 [==============================] - 0s 11ms/step - loss: 5.0025 - accuracy: 0.1440 - lr: 0.0045
Epoch 5/500
4/4 [==============================] - 0s 13ms/step - loss: 3.9631 - accuracy: 0.1000 - lr: 0.0060
Epoch 6/500
4/4 [==============================] - 0s 11ms/step - loss: 3.2777 - accuracy: 0.1160 - lr: 0.0075
Epoch 7/500
4/4 [==============================] - 0s 12ms/step - loss: 2.9017 - accuracy: 0.1280 - lr: 0.0090
Epoch 8/500
4/4 [==============================] - 0s 11ms/step - loss: 2.6063 - accuracy: 0.1360 - lr: 0.0105
Epoch 9/500
4/4 [==============================] - 0s 11ms/step - loss: 2.6977 - accuracy: 0.1240 - lr: 0.0120


In [18]:
pseudo = np.argmax(np.mean(predictions, axis=0), axis=1)
conf = np.max(np.mean(predictions, axis=0), axis=1)
uncert = np.std(predictions, axis=0)
uncert = np.array([uncert[i][pseudo[i]] for i in range(len(pseudo))])

In [19]:
score = conf - uncert
score = (score-min(score))/(max(score)-min(score))
score = score/sum(score)

In [20]:
# indx = np.random.choice(len(score), 50000, p = score)
indx = np.random.choice(len(score), 50000)

In [21]:
X = ubl_train_images[indx]
y = ubl_train_labels[indx]

In [22]:
X = np.concatenate([lbl_train_images, X])
y = np.concatenate([lbl_train_labels, y])

In [23]:
predictions = fit_and_labeling_cnn_13(100, 64)

Epoch 1/100
786/786 [==============================] - 9s 11ms/step - loss: 0.8870 - accuracy: 0.7252 - lr: 0.0300
Epoch 2/100
786/786 [==============================] - 9s 11ms/step - loss: 0.5634 - accuracy: 0.8274 - lr: 0.0015
Epoch 3/100
786/786 [==============================] - 9s 11ms/step - loss: 0.4949 - accuracy: 0.8489 - lr: 0.0030
Epoch 4/100
786/786 [==============================] - 9s 11ms/step - loss: 0.4655 - accuracy: 0.8584 - lr: 0.0045
Epoch 5/100
786/786 [==============================] - 9s 11ms/step - loss: 0.4454 - accuracy: 0.8658 - lr: 0.0060
Epoch 6/100
786/786 [==============================] - 9s 11ms/step - loss: 0.4269 - accuracy: 0.8697 - lr: 0.0075
Epoch 7/100
786/786 [==============================] - 9s 11ms/step - loss: 0.4093 - accuracy: 0.8763 - lr: 0.0090
Epoch 8/100
786/786 [==============================] - 9s 11ms/step - loss: 0.3895 - accuracy: 0.8813 - lr: 0.0105
Epoch 9/100
786/786 [==============================] - 9s 11ms/step - loss: 0.38

In [24]:
pseudo = np.argmax(np.mean(predictions, axis=0), axis=1)
conf = np.max(np.mean(predictions, axis=0), axis=1)
uncert = np.std(predictions, axis=0)
uncert = np.array([uncert[i][pseudo[i]] for i in range(len(pseudo))])

In [25]:
score = conf - uncert
score = (score-min(score))/(max(score)-min(score))
score = score/sum(score)

In [26]:
# indx = np.random.choice(len(score), 200000, p = score)
indx = np.random.choice(len(score), 200000)

In [27]:
X = ubl_train_images[indx]
y = ubl_train_labels[indx]

In [28]:
X = np.concatenate([lbl_train_images, X])
y = np.concatenate([lbl_train_labels, y])

In [ ]:
predictions = fit_and_labeling_cnn_13(100, 64)
# 0.9195 (score 사용)
# 

Epoch 1/100
3129/3129 [==============================] - 35s 11ms/step - loss: 0.2502 - accuracy: 0.9284 - lr: 0.0300
Epoch 2/100
3129/3129 [==============================] - 34s 11ms/step - loss: 0.1866 - accuracy: 0.9455 - lr: 0.0015
Epoch 3/100
3129/3129 [==============================] - 34s 11ms/step - loss: 0.1699 - accuracy: 0.9508 - lr: 0.0030
Epoch 4/100
3129/3129 [==============================] - 34s 11ms/step - loss: 0.1633 - accuracy: 0.9528 - lr: 0.0045
Epoch 5/100
3129/3129 [==============================] - 34s 11ms/step - loss: 0.1573 - accuracy: 0.9543 - lr: 0.0060
Epoch 6/100
3129/3129 [==============================] - 34s 11ms/step - loss: 0.1538 - accuracy: 0.9552 - lr: 0.0075
Epoch 7/100
3129/3129 [==============================] - 34s 11ms/step - loss: 0.1520 - accuracy: 0.9548 - lr: 0.0090
Epoch 8/100
3129/3129 [==============================] - 34s 11ms/step - loss: 0.1480 - accuracy: 0.9558 - lr: 0.0105
Epoch 9/100
3129/3129 [==============================] -